In [1]:
# Relevant imports
from science_jubilee.Machine import Machine
from science_jubilee.decks.Deck import Deck
from science_jubilee.tools.Camera import Camera
from science_jubilee.tools.Pipette import Pipette

In [2]:
# connect to the machine and add a deck
m = Machine(address="jubilee.local")
deck = m.load_deck('lab_automation_deck_MAfeb')

In [3]:
# load tools for this workflow
pipette = Pipette.from_config(1, 'ot', 'P300_config.json')
camera = Camera(3, 'cam')

m.load_tool(pipette)
m.load_tool(camera)

In [4]:
# add labware
tip_rack = deck.load_labware('opentrons_96_tiprack_300ul', 4)
pipette.add_tiprack(tip_rack)

source_wells = deck.load_labware('scienfocus_6_wellplate_15900ul', 0)
destination_wells = deck.load_labware('corning_96_wellplate_360ul_flat', 1)

In [5]:
# pickup the pipette
m.pickup_tool(pipette)

In [6]:
# equip pipette with a fresh tip
pipette.pickup_tip()

In [7]:
# add 50uL of water to every well
water = source_wells["A1"]
all_wells = list(destination_wells.wells.values())
m.deck._safe_z = 20 ## manually lower for speed

pipette.transfer(vol=50, source_well=water, destination_well=all_wells, blowout=True)

In [8]:
# add 100ul of red to row A
red = source_wells["A2"]
row_a = destination_wells.get_row('A')
for d_well in row_a:
    pipette.transfer(vol=100, source_well=red, destination_well=d_well, blowout=True, mix_after=(50,3))

In [ ]:
# Move 100uL from the first well in the column to the next & mix, and so on
m.deck._safe_z = 15 # manually lower for speed
row_count = len(destination_wells.row_data)
for column_id in destination_wells.column_data:
    col = destination_wells.column_data[column_id]
    for num, well in enumerate(col):
        if num + 1 < row_count: # if there is another well to move to
            next_well = col[num+1]
            pipette.transfer(vol=100, source_well=well, destination_well=next_well, blowout=True, mix_after=(50,3))

m.deck._safe_z = 60

In [14]:
# dispose tip
garbage = m.deck.slots_data['-1']["offset"]
garbage = (garbage[0], garbage[1], 10)

In [11]:
pipette.drop_tip(garbage)

In [18]:
# misc
m.deck._safe_z = 65

In [19]:
m.move_to(z=100)

In [ ]:
m.pickup_tool(camera)